                                                 Домашнє завдання N 7

Так як ми працюємо з рекомендованними системами, по завданню нам радять використовувати бібліотеку Surprise, котра якраз є по суті додатком до знайомої нам бібліотеки scikit-learn для тренування моделей рекомендаційних систем.

Тому давайте імпортуємо цю бібліотеку і також завантажимо потрібний датасет movielens на 100 к даних. Думаю, нам буде достатньо цих даних для виконання домашнього завдання: